In [1]:
# Cell 1: Thiết lập và Tải Dữ liệu Đã Xử lý

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# --- XÂY DỰNG ĐƯỜNG DẪN CƠ SỞ ---
# Giả định Notebook đang ở thư mục 'code/'
PROJECT_ROOT = Path('..')
DATA_PATH = PROJECT_ROOT / 'input' / 'process' 

# --- 1. TẢI DỮ LIỆU ĐÃ XỬ LÝ ---
try:
    X_train_processed = pd.read_csv(DATA_PATH / 'X_train_processed.csv')
    y_df = pd.read_csv(DATA_PATH / 'y_train.csv')
    X_test_processed = pd.read_csv(DATA_PATH / 'X_test_processed.csv')
    
    y = y_df['Class']
    
    # Tải Id
    df_test_original = pd.read_csv('../input/test.csv')
    test_id = df_test_original['Id']

    print("Tải dữ liệu đã xử lý thành công!")

except FileNotFoundError as e:
    print(f"LỖI TẢI FILE: {e}. KHÔNG THỂ TIẾP TỤC.")
    raise RuntimeError("Dừng thực thi do FileNotFoundError.") from e

# Tách tập Train/Validation để đánh giá Ensemble trước khi Submission
X_train, X_val, y_train, y_val = train_test_split(
    X_train_processed, 
    y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y 
)

print(f"Kích thước tập huấn luyện: {X_train.shape}")

Tải dữ liệu đã xử lý thành công!
Kích thước tập huấn luyện: (11516, 28)


In [3]:
# Cell 2: Định nghĩa và Huấn luyện Voting Classifier (Soft Voting)

# Định nghĩa các mô hình thành phần (Sử dụng các tham số đã tinh chỉnh trong base_model.ipynb)
# Cần dùng các mô hình hỗ trợ 'predict_proba' cho Soft Voting (loại bỏ SVC và KNN)

clf1 = XGBClassifier(random_state=42, objective='multi:softmax', eval_metric='mlogloss', n_estimators=300, max_depth=7, learning_rate=0.1, use_label_encoder=False, n_jobs=-1)
clf2 = HistGradientBoostingClassifier(random_state=42, max_iter=300, max_depth=10, learning_rate=0.1)
clf3 = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=300, max_depth=10, n_jobs=-1)

# Định nghĩa Voting Classifier (Soft Voting - Kết hợp xác suất)
eclf = VotingClassifier(
    estimators=[('xgb', clf1), ('histgb', clf2), ('rf', clf3)],
    voting='soft', # Soft Voting: Kết hợp xác suất dự đoán (cần predict_proba)
    n_jobs=4
)

print("Bắt đầu huấn luyện Ensemble Model (Voting Classifier)...")

eclf.fit(X_train, y_train)

# Đánh giá Ensemble trên tập Validation
y_pred_ensemble = eclf.predict(X_val)
f1_macro_ensemble = f1_score(y_val, y_pred_ensemble, average='macro')

print("\n--- Đánh giá Ensemble Model ---")
print(f"Macro F1 Score Ensemble (Validation): {f1_macro_ensemble:.4f}")
print("Classification Report:")
print(classification_report(y_val, y_pred_ensemble))

Bắt đầu huấn luyện Ensemble Model (Voting Classifier)...

--- Đánh giá Ensemble Model ---
Macro F1 Score Ensemble (Validation): 0.4448
Classification Report:
              precision    recall  f1-score   support

           0       0.28      0.21      0.24       100
           1       0.01      0.00      0.01       220
           2       0.41      0.39      0.40       204
           3       0.64      0.56      0.60        64
           4       0.40      0.27      0.33        62
           5       0.68      0.71      0.70       231
           6       0.29      0.23      0.26       414
           7       0.74      0.85      0.79        92
           8       0.58      0.55      0.56       297
           9       0.49      0.51      0.50       404
          10       0.44      0.59      0.51       792

    accuracy                           0.46      2880
   macro avg       0.45      0.44      0.44      2880
weighted avg       0.43      0.46      0.44      2880



In [4]:
# Cell 3: Tạo Tệp Gửi (Submission Cuối cùng)

print("\nHuấn luyện Ensemble Model trên toàn bộ tập Train...")

# Huấn luyện lại Ensemble trên toàn bộ X_train_processed
eclf_final = eclf.fit(X_train_processed, y)

# Dự đoán trên tập Test
y_test_pred_ensemble = eclf_final.predict(X_test_processed)

# Tạo DataFrame Submission
submission_df_ensemble = pd.DataFrame({
    'Id': test_id, 
    'Class': y_test_pred_ensemble.astype(int) 
})

# Lưu tệp Submission vào thư mục output/ensemble
OUTPUT_DIR = Path('../output/ensemble/') 
OUTPUT_DIR.mkdir(parents=True, exist_ok=True) # Tạo thư mục output/ensemble/

submission_filename_ensemble = f'submission_Ensemble_F1_{f1_macro_ensemble:.4f}.csv'
submission_df_ensemble.to_csv(OUTPUT_DIR / submission_filename_ensemble, index=False)

print(f"\nĐã tạo tệp submission cuối cùng thành công: {submission_filename_ensemble} trong thư mục output/ensemble/")


Huấn luyện Ensemble Model trên toàn bộ tập Train...

Đã tạo tệp submission cuối cùng thành công: submission_Ensemble_F1_0.4448.csv trong thư mục output/ensemble/
